In [1]:
# import riiideducation
# import dask.dataframe as dd
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error, f1_score, roc_auc_score
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot') #ggplot스타일 사용
plt.rc('axes', unicode_minus=False)

data_file_path = './Riiid_dataset/riiid-test-answer-prediction/'

In [2]:
train = pd.read_csv(data_file_path + 'train.csv',
                   usecols=[1, 2, 3, 4, 5, 7, 8, 9],
                   dtype={'timestamp': 'int64',
                          'user_id': 'int32',
                          'content_id': 'int16',
                          'content_type_id': 'int8',
                          'task_container_id' : 'int16',
                          'answered_correctly':'int8',
                          'prior_question_elapsed_time': 'float32',
                          'prior_question_had_explanation': 'boolean'}
                   )
sub = pd.read_csv(f'{data_file_path}/example_sample_submission.csv')
lec = pd.read_csv(f'{data_file_path}/lectures.csv')
questions_df = pd.read_csv(f'{data_file_path}/questions.csv')
test = pd.read_csv(f'{data_file_path}/example_test.csv')

In [5]:
# content_type_id가 True인 것을 제외
# 1인경우 유저의 question이 담긴 것이 아닌 강의를 단순히 보고 있는 것
train = train[train['content_type_id'] == False].sort_values('timestamp').reset_index(drop = True)

In [6]:
elapsed_mean = train.prior_question_elapsed_time.mean()

In [8]:
train['task_container_id'].value_counts()

14      801910
15      796490
4       692036
5       689967
6       688749
         ...  
9932       173
9999       172
9925       171
9926       170
9937       169
Name: task_container_id, Length: 10000, dtype: int64

In [9]:
group1 = train.loc[(train.content_type_id == False), ['task_container_id', 'user_id']].groupby(['task_container_id']).agg(['count'])
group1.columns = ['avg_questions']
group2 = train.loc[(train.content_type_id == False), ['task_container_id', 'user_id']].groupby(['task_container_id']).agg(['nunique'])
group2.columns = ['avg_questions']
group3 = group1 / group2

In [10]:
group3

,avg_questions
task_container_id,
0,1.006649
1,1.006161
2,1.005353
3,1.004791
4,1.759428
...,...
9995,1.450382
9996,1.400000
9997,1.401515


In [11]:
# 데이터 이동평균 해볼 것
group3['avg_questions_seen'] = group3.avg_questions.cumsum()